### Import libraries

In [ ]:
# Import necessary libraries
import os

# Import necessary libraries for data processing
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Import necessary libraries for model
from keras.callbacks import Callback
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from keras.metrics import Precision, Recall, F1Score, AUC

# Import necessary libraries for evaluation
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


## Directories

In [ ]:
trade_data_dir = 'trade_data__A.csv'

Save_model_dir = 'LSTM_model'

# Data Preparation

### Load and merge data

In [ ]:
# List of stocks and splits
stocks = ['A', 'B', 'C', 'D', 'E']
splits = ['0', '1', '2', '3', '4','5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15']

# Load the data from a CSV file
market_data_list = []
trade_data_list = []

# loop over the stocks and load the data
for stock in stocks:

    trade_data_dir = f'trade_data_{stock}.csv'
    if os.path.exists(trade_data_dir):
        trade_data = pd.read_csv(trade_data_dir)
        trade_data['stock'] = stock
        trade_data_list.append(trade_data)
    else:
        print(f"File {trade_data_dir} does not exist")
        continue

    for split in splits:
        market_data_dir = f'market_data_{stock}_{split}.csv'

        if os.path.exists(market_data_dir):
            market_data = pd.read_csv(market_data_dir)

            # Add stock identifier
            market_data['stock'] = stock
            market_data_list.append(market_data)
        else:
            print(f"File {market_data_dir} does not exist")


# Combine market and trade data for the period
period_market_data = pd.concat(market_data_list, ignore_index=True)
period_trade_data = pd.concat(trade_data_list, ignore_index=True)

# Merge the data on the nearest timestamp
period_market_data['timestamp'] = pd.to_datetime(period_market_data['timestamp'])
period_trade_data['timestamp'] = pd.to_datetime(period_trade_data['timestamp'])

# Sort values for merge_asof
period_market_data.sort_values(by=['stock', 'timestamp'], inplace=True)
period_trade_data.sort_values(by=['stock', 'timestamp'], inplace=True)

# Merge market and trade data
merged_data = pd.merge_asof(
    period_market_data,
    period_trade_data,
    on='timestamp',
    by='stock',
    direction='nearest'
)

<ipython-input-26-cee42e615d8a>:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  market_data['timestamp'] = pd.to_datetime(market_data['timestamp'])
<ipython-input-26-cee42e615d8a>:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trade_data['timestamp'] = pd.to_datetime(trade_data['timestamp'])


### Feature Engineering

In [ ]:
# Calculate the spread between the market and trade data
merged_data['spread'] = merged_data['askPrice'] - merged_data['bidPrice']

# Add a moving average of the spread
merged_data['bidPrice_ma'] = merged_data['bidPrice'].rolling(window=5).mean()
merged_data['askPrice_ma'] = merged_data['askPrice'].rolling(window=5).mean()

# Fill any remaining NANS by backfilling
merged_data.fillna(method='bfill', inplace=True)

<ipython-input-27-70c1d597eccc>:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='bfill', inplace=True)
<ipython-input-27-70c1d597eccc>:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_data.fillna(method='bfill', inplace=True)


### Define the feature set and target set

In [ ]:
# Define the feature and it's fitted scaller
feature_scaler = MinMaxScaler()
features = [
    'bidVolume',
    'bidPrice',
    'askVolume',
    'askPrice',
    'spread',
    'bidPrice_ma',
    'askPrice_ma'
    ]
feature_scaler.fit(merged_data[features])

# Defien the target scaller adn fit it
target_scaler = MinMaxScaler()
target_scaler.fit(merged_data[['bidPrice']])

MinMaxScaler()

### Create a scaled array

In [ ]:
x_data = merged_data[features].values
y_data = merged_data[['bidPrice']].values

# Scale the data
x_data_scaled = feature_scaler.transform(x_data)
y_data_scaled = target_scaler.transform(y_data)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


### Define input and output

In [ ]:
# Define timesteps
timesteps = 10

# Prepare input (x) and output (y)
x = []
y = []

for i in range(len(merged_data) - timesteps):
    x.append(x_data_scaled[i:i + timesteps])
    y.append(y_data_scaled[i + timesteps])

x = np.array(x)
y = np.array(y)

### Split data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Implementation of Model

### Define Model class

In [ ]:
# reshape the input data
input_shape = (x_train.shape[1], x_train.shape[2])

# define the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=input_shape, return_sequences=False))
model.add(Dense(1, activation='linear')) # TODO:Adjust the activation function to sigmoid if needed

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Compile model

In [ ]:
# compile the model for training
model.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['mae']
    ) # TODO:Adjust the loss function to binary_crossentropy if needed

### Train and validate model

In [ ]:

# # train the model
# model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test))


# Custom callback to save and print metrics at each epoch
class EpochLogger(Callback):
    def __init__(self):
        super().__init__()
        self.epochs = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        # Print the metrics for the current epoch
        print(f"Epoch {epoch + 1}: {logs}")

        # Save metrics in a list for later use
        self.epochs.append({"epoch": epoch + 1, **logs})

# Instantiate the custom callback
epoch_logger = EpochLogger()

# Train the model and use the custom callback
history = model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_data=(x_test, y_test),
    callbacks=[epoch_logger]
)

# Save the epoch logs to a CSV file
epoch_logs_df = pd.DataFrame(epoch_logger.epochs)
epoch_logs_df.to_csv('epoch_logs.csv', index=False)

# Print the saved dataframe
print(epoch_logs_df)

Epoch 1/20
2495/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0045 - mae: 0.0244Epoch 1: {'loss': 0.0009458701242692769, 'mae': 0.010099845007061958, 'val_loss': 7.651151099707931e-05, 'val_mae': 0.0053542610257864}
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 27s 9ms/step - loss: 0.0045 - mae: 0.0243 - val_loss: 7.6512e-05 - val_mae: 0.0054
Epoch 2/20
2496/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7.8146e-05 - mae: 0.0055Epoch 2: {'loss': 7.735005056019872e-05, 'mae': 0.005493850447237492, 'val_loss': 7.575220661237836e-05, 'val_mae': 0.005548417568206787}
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - loss: 7.8144e-05 - mae: 0.0055 - val_loss: 7.5752e-05 - val_mae: 0.0055
Epoch 3/20
2498/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7.4957e-05 - mae: 0.0054Epoch 3: {'loss': 7.388467201963067e-05, 'mae': 0.005389463156461716, 'val_loss': 7.47006997698918e-05, 'val_mae': 0.005700130946934223}
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 42s 10ms/step - loss: 7.4956e-05 - mae: 0.0054 - val_loss: 7.4701e-05 - 

### Test model

In [36]:
# Predict on the scaled test data
y_pred_scaled = model.predict(x_test)

# Apply inverse transformation to scale back to the original range and inverse transform y_test
y_pred_original = target_scaler.inverse_transform(y_pred_scaled)
y_test_original = target_scaler.inverse_transform(y_test)

print("Original Test Values (unscaled):", y_test_original.ravel()[:10])
print("Predicted Values (unscaled):", y_pred_original.ravel()[:10])

625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


ValueError: Found array with dim 3. None expected <= 2.

## Other resources code

### Load model

In [ ]:
loaded_model = load_model(Save_model_dir)

### Save model

In [ ]:
# save in tensoflow model
model.save(Save_model_dir)